In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from io import StringIO
import time

/Users/minh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [8]:
data = requests.get(standings_url)

In [9]:
soup = bs(data.text)

In [10]:
standings_table = soup.select("table.stats_table")[0]

In [11]:
links = standings_table.find_all("a")

In [12]:
links = [link.get("href") for link in links]

In [13]:
links = [link for link in links if "/squads/" in link]

In [14]:
team_urls = [f"https://fbref.com{link}" for link in links]

In [15]:
team_url = team_urls[0]
data = requests.get(team_url)

In [16]:
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")

In [17]:
soup = bs(data.text)

In [18]:
links = soup.find_all("a")

In [19]:
links = [link.get("href") for link in links]

In [20]:
links = [link for link in links if link and "all_comps/shooting/" in link]

In [21]:
data = requests.get(f"https://fbref.com{links[0]}")

In [22]:
shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]

In [23]:
shooting.columns = shooting.columns.droplevel()

In [24]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PKatt"]], on="Date")

In [25]:
years = list(range(2022, 2020, -1))
years

[2022, 2021]

In [26]:
all_matches = []

In [27]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [28]:

for year in years:
    data = requests.get(standings_url)
    soup = bs(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = [link.get("href") for link in standings_table.find_all('a')]
    links = [link for link in links if '/squads/' in link]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

        soup = bs(data.text)
        link = [link.get("href") for link in soup.find_all("a")]
        links = [link for link in links if link and "all_comps/shooting/" in link]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

        time.sleep(2)

IndexError: list index out of range

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [column.lower() for column in match_df.columns]

In [ ]:
match_df.to_csv("matches.csv")